# Simple Agentic Tutorial with Single Tool Use

This tutorial is designed to give new users an overview of how to use **Simple Agentic** effectively. It demonstrates basic single-tool usage and provides guidance on switching between a remote cluster and a local Podman setup.

## Overview

Agentic is a framework that allows users to interact with AI agents in a structured and efficient manner. This tutorial walks you through the following steps:
1. Setting up the environment.
2. Configuring the agent with a single tool.
3. Running queries and interpreting responses.
4. Switching between a remote cluster and a local Podman setup.

## Prerequisites

Before starting, ensure you have the following:
- Python.
- Access to a remote cluster or a local Podman setup.
- API keys and environment variables configured (e.g., `INFERENCE_MODEL`, `TAVILY_SEARCH_API_KEY`, `LLAMA_STACK_ENDPOINT`, `LLAMA_STACK_PORT`).


### 1. Setting Up the Environment
- Import the necessary libraries.
- Load environment variables.

In [ ]:
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client import LlamaStackClient
from termcolor import cprint

inference_model = "ibm-granite/granite-3.2-8b-instruct"
tavily_search_api_key = "YOUR_TAVILY_API_KEY"
endpoint = "your-llama-stack-endpoint" # Replace with your Llama Stack endpoint
port = 8321
environment = "nerc" # Use the `environment` variable to toggle between a remote cluster (`nerc`) and a local Podman setup (`local`).
print(f"Model: {inference_model}")

Model: ibm-granite/granite-3.2-8b-instruct


### Client Initialization
- Initialize the `LlamaStackClient` with the appropriate base URL and provider data.

In [2]:
client =  LlamaStackClient(
        base_url=f"http://localhost:{port}" if environment=="local" else endpoint,
        provider_data = {"tavily_search_api_key": tavily_search_api_key}
)

### Configuring the Agent
- Create an `Agent` instance with the desired model, instructions, tools.

In [3]:
agent = Agent(
    client, 
    model=inference_model,
    instructions="""You are a helpful websearch assistant. When you are asked to search the web you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    tool_config={"tool_choice":"required"},
    sampling_params={"max_tokens":4096}
)

### Set User Queries
- Define user prompts to interact with the agent.

In [4]:
user_prompts = [
    "What’s latest in OpenShift?",
]

### Query execution
- Use the agent to create a session and process user queries.
- Log and display the agent's responses for each query.

In [5]:
session_id = agent.create_session("test-session")
for prompt in user_prompts:
    cprint(f"User> {prompt}", "green")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

User> What’s latest in OpenShift?
inference> <tool_call>
tool_execution> Tool:brave_search Args:{'query': 'latest updates on OpenShift'}
tool_execution> Tool:brave_search Response:{"query": "latest updates on OpenShift", "top_k": [{"title": "What's new in OpenShift? - Red Hat Developer", "url": "https://developers.redhat.com/products/openshift/whats-new-red-hat-openshift", "content": "Please see the release notes to discover all of the updates. Get started with Red Hat OpenShift 4.18. To get started with Red Hat OpenShift 4.18, try the Developer Sandbox. It provides free access to a shared OpenShift cluster for your development needs without requiring you to set up and configure an OpenShift cluster.", "score": 0.756376, "raw_content": null}, {"title": "Red Hat Blends Innovation with Stability in Latest Release of Red Hat ...", "url": "https://www.redhat.com/en/about/press-releases/red-hat-blends-innovation-stability-latest-release-red-hat-openshift", "content": "Red Hat, Inc., the wor

## Key Takeaways
- This tutorial demonstrates how to set up and use a single tool with Agentic.
- It highlights the flexibility of switching between remote and local environments.
- By following these steps, users can quickly get started with Agentic and explore its capabilities.

For more advanced use cases, please check our other example jupyter notebooks.